## Installing the dependencies

In [ ]:
!pip install semantic-kernel==0.3.1.dev0
!pip install pyowm

In [2]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureTextCompletion
from semantic_kernel.planning.basic_planner import BasicPlanner
from semantic_kernel.skill_definition import sk_function
from pyowm import OWM
from datetime import datetime, timedelta


## Defining our native functions

In [3]:
class WeatherPlugin:
    def __init__(self) -> None:
        self.mgr = OWM("API_KEY").weather_manager()

    @sk_function(
        description="Predict weather forecast only information from a given location",
        name="ForecastAtPlace"
    )
    def get_weather_forecast(self, location : str) -> str:
        forecast = self.mgr.forecast_at_place(location, '3h')
        target_time = datetime.now() + timedelta(hours=6)
        weather_at_target_time = forecast.get_weather_at(target_time)
        temperature = weather_at_target_time.temperature('celsius'),
        status = weather_at_target_time.detailed_status
        rain = weather_at_target_time.rain
        humidity = weather_at_target_time.humidity
        ret = {
            'location': location,
            'current_time':datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'forecast_time': target_time.strftime('%Y-%m-%d %H:%M:%S'),
            'temperature': temperature,
            'status': status,
            'rain': rain,
            'humidity': humidity
        }
        return str(ret)

    @sk_function(
        description="Generate current weather information only from a given location[NOT FORECASTS]. The returned information includes a detailed status, wind, humidity, temperature in celcius, rain, and clouds",
        name="WeatherAtPlace"
    )
    def weather_at_place(self, location: str) -> str:
        observation = self.mgr.weather_at_place(location)
        w = observation.weather
        ret = {
            'current_time' : datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'detailed_status' : w.detailed_status,
            'wind' : w.wind(),
            'humidity' : w.humidity,
            'temperature' : w.temperature('celsius'),
            'rain' : w.rain,
            'heat_index' : w.heat_index,
            'clouds' : w.clouds
        }
        return str(ret)

## Initializing the Kernel with Text Davinci as our completion service.

In [6]:
kernel = sk.Kernel()
planner = BasicPlanner()

# ENV VARIABLES
# AZURE_OPENAI_DEPLOYMENT_NAME="Text-Davinci"
# AZURE_OPENAI_ENDPOINT="<ENDPOINT>"
# AZURE_OPENAI_API_KEY="<API_KEY>"

deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
kernel.add_text_completion_service("dv", AzureTextCompletion(deployment, endpoint, api_key))

kernel.import_skill(WeatherPlugin(), skill_name="WeatherPlugin")

## Making our semantic functions.

In [ ]:
WeatherReporter = kernel.create_semantic_function(
    """You are a weather reporter. you will receive the weather data as json, and you will tell it to your viewers in a concise, catchy and jolly manner. mention times if present in data.
----------------------
{{$input}}
""",
    description="Receives weather data as json, and converts it to human readable form.",
    skill_name="WeatherReporter",
    max_tokens=1000,
    temperature=0.3,
    top_p=0.5)

WeatherForecaster = kernel.create_semantic_function(
    """You are a weather reporter who reports forecasts. you will receive the weather forecasts as json, and you will tell it to your viewers in a concise, catchy and jolly manner. make sure to mention that it is a forecast, and mention times if present in data.
----------------------
{{$input}}
""",
    description="Receives weather forecast information as json, and converts it to human readable form.",
    skill_name="WeatherForecaster",
    max_tokens=1000,
    temperature=0.3,
    top_p=0.5)

## Generating a plan for current weather in New York

In [13]:
ask = "What is it like in New York?"
plan = await planner.create_plan_async(ask, kernel) # This only works in a notebook. use asyncio.run in a local python environment
print(plan.generated_plan)

    {
        "input": "New York",
        "subtasks": [
            {"function": "WeatherPlugin.WeatherAtPlace"},
            {"function": "WeatherReporter.f_afe25724_d54d_4af9_93cb_dbf0c7f1e625"}
        ]
    }


In [15]:
out = await planner.execute_plan_async(plan, kernel)
print(out)


Good morning everyone! It's 7:23 am on this beautiful Wednesday morning and the weather is looking a bit overcast. The temperature is a pleasant 21.68 degrees Celsius with a maximum of 22.79 and a minimum of 20.08. The humidity is 81% and the wind is blowing at 3.13 km/h with gusts of up to 5.36 km/h. The sky is completely covered in clouds, so don't forget your umbrella if you're heading out!


## Making a plan for forecast in Lahore

In [18]:
ask = "What is the forecast prediction for Lahore?"
plan = await planner.create_plan_async(ask, kernel) # This only works in a notebook. use asyncio.run in a local python environment
print(plan.generated_plan)

    {
        "input": "Lahore",
        "subtasks": [
            {"function": "WeatherPlugin.ForecastAtPlace"},
            {"function": "WeatherForecaster.f_2ee3a48b_69f0_4d75_b046_e2b6339f15fc"}
        ]
    }


In [19]:
out = await planner.execute_plan_async(plan, kernel)
print(out)


Good morning, Lahore! It's currently 7:26 AM and the forecast for 1:26 PM is sunny with a high of 39 degrees Celsius and a low of 37 degrees Celsius. It will feel like 44 degrees Celsius and the humidity will be 46%. Enjoy your day!
